<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/DL_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision
import torch  
import numpy as np

In [ ]:
def sgd_factorise (A, rank, num_epochs =1000,lr =0.01):
  U = torch.rand(A.shape[0], rank)
  V = torch.rand(A.shape[1], rank)
  for n in range(num_epochs): 
    for r in range(A.shape[0]):
      for c in range(A.shape[1]): 
        e = A[r][c] - U[r]@V[c].t()
        Ur_origin = U[r] 
        U[r] = U[r] + lr * e * V[c]
        V[c] = V[c] + lr * e * Ur_origin
  return U,V

In [ ]:
A_array = np.array([0.3374,0.6005,0.1735,3.3359, 0.0492,1.8374,2.9407,0.5301,2.2620]).reshape(3,3)
A = torch.from_numpy(A_array) 
U,V = sgd_factorise(A, 2)
print (U,V)
print(torch.nn.functional.mse_loss(U@V.t(), A, reduction ='sum'))

tensor([[-0.0856,  0.6243],
        [ 1.7237,  0.2428],
        [ 1.3505,  0.9830]]) tensor([[ 1.8072,  0.5980],
        [-0.1193,  0.7806],
        [ 1.0350,  0.7324]])
tensor(0.1226, dtype=torch.float64)


In [ ]:
U,S,V = torch.svd(A)
S[-1] = 0
print(torch.nn.functional.mse_loss(U@torch.diag(S)@V.t(), A, reduction ='sum'))

tensor(0.1219, dtype=torch.float64)


In [ ]:
def sgd_factorise_masked (A, M, rank, num_epochs =1000,lr =0.01):
  U = torch.rand(A.shape[0], rank)
  V = torch.rand(A.shape[1], rank)
  for n in range(num_epochs): 
    for r in range(A.shape[0]):
      for c in range(A.shape[1]):
        if M[r,c] == 1:
          e = A[r][c] - U[r]@V[c].t()
          Ur_origin = U[r] 
          U[r] = U[r] + lr * e * V[c]
          V[c] = V[c] + lr * e * Ur_origin
  return U,V

In [ ]:
A_origin = torch.from_numpy(np.array([0.3374,0.6005,0.1735,3.3359, 0.0492,1.8374,2.9407,0.5301,2.2620]).reshape(3,3))
A_array = np.array([0.3374,0.6005,0.1735,np.nan, 0.0492,1.8374,2.9407,np.nan,2.2620]).reshape(3,3)
A = torch.from_numpy(A_array) 
M_array = np.array ([1,1,1,0,1,1,1,0,1]).reshape(3,3)
M = torch.from_numpy(M_array)
U,V = sgd_factorise_masked(A, M, 2, num_epochs =1000)
A_completion = U@V.t()
print(U,V)
print(A_completion)
print(torch.nn.functional.mse_loss(A_completion, A_origin, reduction ='sum'))

tensor([[ 0.6466, -0.4184],
        [ 0.8007,  1.1769],
        [ 1.5570,  0.9241]]) tensor([[ 1.2362,  1.0987],
        [ 0.6632, -0.4093],
        [ 0.8844,  0.9589]])
tensor([[0.3397, 0.6001, 0.1707],
        [2.2830, 0.0493, 1.8367],
        [2.9401, 0.6543, 2.2630]])
tensor(1.1241, dtype=torch.float64)
